In [10]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import spacy
from sklearn.feature_extraction.text import CountVectorizer

# Fungsi untuk memetakan sentimen
def map_sentiment(sentiment):
    if sentiment == 1:
        return 'positif'
    elif sentiment == 0:
        return 'netral'
    else:
        return 'negatif'

# Membaca data CSV (pastikan path file CSV sesuai)
df = pd.read_csv('../sentimen_pemilu_manual_1500.csv')

# Memetakan sentimen menggunakan fungsi map_sentiment
df['sentimen'] = df['sentimen'].map(map_sentiment)

# Menggabungkan semua konten berdasarkan sentimen
sentiment_text = {}
for sentiment in df['sentimen'].unique():
    sentiment_text[sentiment] = " ".join(df[df['sentimen'] == sentiment]['content'])

# Inisialisasi spaCy dengan model bahasa Indonesia
nlp = spacy.load('id_core_news_sm')

# Fungsi untuk preprocessing teks menggunakan spaCy
def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if token.is_alpha]  # Hanya token yang alfanumerik
    return tokens

# Membuat WordCloud untuk setiap sentimen dengan bigram dan kata kerja (verb)
for sentiment, text in sentiment_text.items():
    # Tokenisasi dan preprocessing teks dalam bahasa Indonesia menggunakan spaCy
    tokens = preprocess_text(text)

    # Ekstraksi bigram menggunakan CountVectorizer
    bigram_vectorizer = CountVectorizer(ngram_range=(2, 2), stop_words=None, analyzer='word')
    bigram_counts = bigram_vectorizer.fit_transform(tokens)

    # Mendapatkan kata kerja (verb) dari bigram
    verbs = [word for word in bigram_vectorizer.get_feature_names() if ' ' in word and word.split()[1] in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']] 

    # Memfilter bigram yang mengandung kata kerja (verb)
    verb_bigrams = [bigram for bigram in bigram_vectorizer.get_feature_names_out() if any(verb in bigram.split() for verb in verbs)]

    # Membuat WordCloud dari bigram yang mengandung kata kerja (verb)
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(" ".join(verb_bigrams))

    # Menampilkan WordCloud
    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(sentiment.capitalize() + ' Sentiment (Verb Bigrams)')
    plt.axis("off")
    plt.show()


OSError: [E050] Can't find model 'id_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.